# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments           Customer  \
0  Jun 27 2022  9:01AM  240467        10  0085799484  ISDIN Corporation   
1  Jun 27 2022  9:01AM  240467        10  0085799485  ISDIN Corporation   
2  Jun 27 2022  9:01AM  240467        10  0085799488  ISDIN Corporation   
3  Jun 27 2022  9:01AM  240467        10  0085799490  ISDIN Corporation   
4  Jun 27 2022  9:01AM  240467        10  0085799494  ISDIN Corporation   
5  Jun 27 2022  9:01AM  240467        10  0085799496  ISDIN Corporation   
6  Jun 27 2022  9:01AM  240467        10  0085799492  ISDIN Corporation   
7  Jun 27 2022  9:01AM  240467        10  0085799498  ISDIN Corporation   
8  Jun 27 2022  9:01AM  240467        10  0085799503  ISDIN Corporation   
9  Jun 27 2022  9:01AM  240467        10  0085799502  ISDIN Corporation   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
9   240463       Released         41
10  240464       Released          1
11  240466       Released         58
12  240467       Released         49
13  240469       Released          3

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
240463                6.0      41.0
240464                NaN       1.0
240466                NaN      58.0
240467                NaN      49.0
240469                NaN       3.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
240454               12.0      20.0
240455                0.0       4.0
240456                4.0       0.0
240459                0.0      48.0
240460                0.0      51.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
240454                 12        20
240455                  0         4
240456                  4         0
240459                  0        48
240460                  0        51

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               240454         12        20
1               240455          0         4
2               240456          4         0
3               240459          0        48
4               240460          0        51

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing Released
0               240454        12       20
1               240455                  4
2               240456         4         
3               240459                 48
4               240460                 51

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                      Customer
0    Jun 27 2022  9:01AM  240467        10             ISDIN Corporation
49   Jun 27 2022  9:01AM  240466        10             ISDIN Corporation
107  Jun 27 2022  8:58AM  240469        10  Snap Medical Industries, LLC
110  Jun 27 2022  8:31AM  240459        10             ISDIN Corporation
158  Jun 27 2022  8:31AM  240464        12                     Hush Hush
159  Jun 27 2022  8:30AM  240460        10             ISDIN Corporation
210  Jun 27 2022  8:30AM  240463        12                     Hush Hush
257  Jun 27 2022  8:30AM  240462        10             ISDIN Corporation
293  Jun 27 2022  8:29AM  240461        10             ISDIN Corporation
347  Jun 27 2022  8:03AM  240456        10             ISDIN Corporation

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                      Customer  \
0  Jun 27 2022  9:01AM  240467        10             ISDIN Corporation   
1  Jun 27 2022  9:01AM  240466        10             ISDIN Corporation   
2  Jun 27 2022  8:58AM  240469        10  Snap Medical Industries, LLC   
3  Jun 27 2022  8:31AM  240459        10             ISDIN Corporation   
4  Jun 27 2022  8:31AM  240464        12                     Hush Hush   
5  Jun 27 2022  8:30AM  240460        10             ISDIN Corporation   
6  Jun 27 2022  8:30AM  240463        12                     Hush Hush   
7  Jun 27 2022  8:30AM  240462        10             ISDIN Corporation   
8  Jun 27 2022  8:29AM  240461        10             ISDIN Corporation   
9  Jun 27 2022  8:03AM  240456        10             ISDIN Corporation   

  Executing Released  
0                 49  
1                 58  
2                  3  
3                 48  
4                  1  
5                 51  
6         6       41  
7                 36  
8                 54  
9         4

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                      Customer  \
0  Jun 27 2022  9:01AM  240467        10             ISDIN Corporation   
1  Jun 27 2022  9:01AM  240466        10             ISDIN Corporation   
2  Jun 27 2022  8:58AM  240469        10  Snap Medical Industries, LLC   
3  Jun 27 2022  8:31AM  240459        10             ISDIN Corporation   
4  Jun 27 2022  8:31AM  240464        12                     Hush Hush   
5  Jun 27 2022  8:30AM  240460        10             ISDIN Corporation   
6  Jun 27 2022  8:30AM  240463        12                     Hush Hush   
7  Jun 27 2022  8:30AM  240462        10             ISDIN Corporation   
8  Jun 27 2022  8:29AM  240461        10             ISDIN Corporation   
9  Jun 27 2022  8:03AM  240456        10             ISDIN Corporation   

  Released Executing  
0       49            
1       58            
2        3            
3       48            
4        1            
5       51            
6       41         6  
7       36            
8       54            
9                  4

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                      Customer  \
0  Jun 27 2022  9:01AM  240467        10             ISDIN Corporation   
1  Jun 27 2022  9:01AM  240466        10             ISDIN Corporation   
2  Jun 27 2022  8:58AM  240469        10  Snap Medical Industries, LLC   
3  Jun 27 2022  8:31AM  240459        10             ISDIN Corporation   
4  Jun 27 2022  8:31AM  240464        12                     Hush Hush   

  Released Executing  
0       49            
1       58            
2        3            
3       48            
4        1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse                      Customer  \
0  Jun 27 2022  9:01AM  240467        10             ISDIN Corporation   
1  Jun 27 2022  9:01AM  240466        10             ISDIN Corporation   
2  Jun 27 2022  8:58AM  240469        10  Snap Medical Industries, LLC   
3  Jun 27 2022  8:31AM  240459        10             ISDIN Corporation   
4  Jun 27 2022  8:31AM  240464        12                     Hush Hush   

   Released  Executing  
0      49.0        NaN  
1      58.0        NaN  
2       3.0        NaN  
3      48.0        NaN  
4       1.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                      Customer  \
0  Jun 27 2022  9:01AM  240467        10             ISDIN Corporation   
1  Jun 27 2022  9:01AM  240466        10             ISDIN Corporation   
2  Jun 27 2022  8:58AM  240469        10  Snap Medical Industries, LLC   
3  Jun 27 2022  8:31AM  240459        10             ISDIN Corporation   
4  Jun 27 2022  8:31AM  240464        12                     Hush Hush   

   Released  Executing  
0      49.0        0.0  
1      58.0        0.0  
2       3.0        0.0  
3      48.0        0.0  
4       1.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         1923700     299.0        4.0
12          480927      42.0        6.0
19          480909      24.0       12.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  1923700     299.0        4.0
1        12   480927      42.0        6.0
2        19   480909      24.0       12.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10     299.0        4.0
1        12      42.0        6.0
2        19      24.0       12.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse     Status  Value
0        10   Released  299.0
1        12   Released   42.0
2        19   Released   24.0
3        10  Executing    4.0
4        12  Executing    6.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10    12    19
Status                      
Executing    4.0   6.0  12.0
Released   299.0  42.0  24.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10    12    19
0          Executing    4.0   6.0  12.0
1           Released  299.0  42.0  24.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10    12    19
0  Executing    4.0   6.0  12.0
1   Released  299.0  42.0  24.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()